In [1]:
import pathlib

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tifffile as tf  # write tiff files
from PIL import Image  # read tiff files
from toml import load
from tqdm import tqdm  # progress bar

In [2]:
CELL_TYPE = "PBMC"

In [3]:
path = pathlib.Path(
    "../../../8.cytopick_analysis/results/PBMC/single_cell_predictions.parquet"
).resolve(strict=True)

# read in the data
df = pd.read_parquet(path)
# show all columns
pd.set_option("display.max_columns", None)
df.head()

,apoptosis_prob,control_prob,pyroptosis_prob,label_true,label_pred,data_split,shuffle,class_name,index,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,Metadata_inducer2,Metadata_inducer2_concentration,Metadata_inducer2_concentration_unit,Metadata_ImageNumber,Metadata_Plate,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Treatment,Metadata_Dose,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMaximum_Y,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_X,Metadata_Cytoplasm_AreaShape_BoundingBoxMinimum_Y,Metadata_Site,labels,correct,image_path,image_crop_path
0,0.060589,0.489427,0.449984,1,1,train,True,healthy,2466219.0,PBMC,G02,51149.0,6.0,Disulfiram,1.000,µM,LPS,10.000,µg_per_ml,None,nan,None,190.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,1116.0,1116.0,1117.0,1117.0,LPS,10.000_µg_per_ml,1188.135802,685.890947,1205.0,699.0,1174.0,674.0,14.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
1,0.037797,0.543169,0.419034,1,1,train,True,healthy,992552.0,PBMC,B11,50923.0,6.0,DMSO,0.025,%,LPS,100.000,µg_per_ml,Nigericin,1.000,µM,148.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,1170.0,1170.0,1172.0,1172.0,LPS_Nigericin,100.000_µg_per_ml_1.000_µM,1628.743003,785.290076,1642.0,796.0,1617.0,770.0,4.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
2,0.009804,0.575268,0.414929,1,1,train,True,healthy,7219928.0,PBMC,N02,47664.0,6.0,Disulfiram,1.000,µM,LPS,1.000,µg_per_ml,Nigericin,10.000,µM,16.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,938.0,938.0,943.0,943.0,LPS_Nigericin,1.000_µg_per_ml_10.000_µM,243.827485,595.397661,261.0,609.0,234.0,582.0,16.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
3,0.038737,0.550273,0.410990,1,1,train,True,healthy,4386044.0,PBMC,H09,43534.0,6.0,Disulfiram,2.500,µM,LPS,10.000,µg_per_ml,None,nan,None,114.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,1725.0,1725.0,1729.0,1729.0,LPS,10.000_µg_per_ml,1101.694268,1133.617834,1114.0,1146.0,1089.0,1122.0,2.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...
4,0.019823,0.570437,0.409740,1,1,train,True,healthy,4392343.0,PBMC,H09,43534.0,6.0,Disulfiram,2.500,µM,LPS,10.000,µg_per_ml,None,nan,None,115.0,*70117_20230210MM1_Gasdermin514_CP_BC430856,2500.0,2500.0,2504.0,2504.0,LPS,10.000_µg_per_ml,460.834684,1496.259319,478.0,1509.0,444.0,1484.0,3.0,pyroptosis,True,/home/lippincm/Documents/4TB/data/Interstellar...,/home/lippincm/Documents/4TB/data/Interstellar...


In [4]:
# loop through each image and assign a scale bar
for image_path in df["image_crop_path"]:
    print(image_path)
    # open the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # image to array
    image_array = np.array(image)
    resolution = 3.3446  # pixels per micrometer
    # get the image size
    image_size = image_array.shape
    if image_array.shape[1] <= 100:
        scale_bar_length = 5  # um
        scale_bar_height = 1  # pixels
        padding = 3  # pixels
    elif image_array.shape[1] > 100:
        scale_bar_length = 100
        scale_bar_height = 10
        padding = 10
    # get the bottom right most corner based on scale 1 % of pixels
    scale_bar_x = image_size[1] - (scale_bar_length * resolution) - padding - padding
    scale_bar_y = image_size[0] - (scale_bar_height) - padding
    print(scale_bar_x, scale_bar_y)
    # draw the scale bar
    new = cv2.rectangle(
        image_array,
        (int(scale_bar_x), int(scale_bar_y)),
        (image_size[1] - padding, image_size[0] - padding),
        (255, 255, 255),
        -1,
    )
    # save the image with the scale bar via cv2
    cv2.imwrite(image_path, new)
#

/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_0.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_1.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_2.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_3.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_image_crop_cell_4.png
77.277 96
/home/lippincm/Documents/4TB/data/Interstellar_Analysis/8.cytopick_analysis/figures/PBMC/pyroptosis_shuffled_train_df_im1_im3_im4_composite_